In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import json
import plotly.express as px
import geopandas as gpd

init_notebook_mode(connected=True)  

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
#---------------------------------------------------------------------------------------------------------------------
# ENCOUNTER DATA CLEANING
#---------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------
# HEAR ABOUT DATA CLEANING
#---------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------
# REFERRALS DATA CLEANING
#---------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------
# COUNTY CENSUS DATA CLEANING 
#---------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------
# ZIP CENSUS DATA CLEANING
#---------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------
# TENNESSEE DEPARTMENT OF HEALTH DATA CLEANING
#---------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------
# ADDED LISTS 
#---------------------------------------------------------------------------------------------------------------------
#List to filter out the tennessee counties
counties_to_show = ['Coffee County', 'Franklin County', 'Moore County', 'Bedford County', 'Cannon County', 'Warren County', 'Grundy County', 'Marion County']

# Create the Dash application
app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=[dbc.themes.BOOTSTRAP])

#---------------------------------------------------------------------------------------------------------------------
# SECTION FOR IMPORTED CSV AND SHAPE FILES
#---------------------------------------------------------------------------------------------------------------------

# Import County Shape Files
zip_shape = gpd.read_file('filtered_zip_shapefile.shp')
zip_shape.set_index('zip')
zip_shape = zip_shape.to_crs(epsg = 4326)

#Import Census Files
gen_census = pd.read_csv('01general_census.csv')
age_census = pd.read_csv('01age_census.csv')
emp_census = pd.read_csv('01employment_census.csv')
dis_census = pd.read_csv('01disability_census.csv')
pov_census = pd.read_csv('01poverty_census.csv')

#Import Hear About Data
hear = pd.read_csv('FinalHearAbout.csv')

#Import Condition Encounter Data
ctrends=pd.read_csv('EncounterCondition.csv')
#Groupby 'enc year' and 'enc month' and count'patientid'
monthly_visits = ctrends.groupby(['enc year', 'enc month'])['patientid'].count().reset_index()
years = monthly_visits['enc year'].unique()
#Convert 'enc month' column to numeric
ctrends['enc month'] = pd.to_numeric(ctrends['enc month'])
#Combine 'enc year' and 'enc month'
ctrends['enc date'] = pd.to_datetime(ctrends['enc year'].astype(str) + '-' + ctrends['enc month'].astype(str), format='%Y-%m')
#Groupby 'enc date' and count 'patientid'
year_trends = ctrends.groupby('enc date')['patientid'].count().reset_index()
#Resample to monthly frequency
year_trends = year_trends.resample('M', on='enc date').sum()
#Define bin ranges
bin_ranges = list(range(0, 451, 50))

#---------------------------------------------------------------------------------------------------------------------
# SECTION FOR THE SIDE BAR
#---------------------------------------------------------------------------------------------------------------------
#app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
#Styling sidebar

SIDEBAR_STYLE= {
    "position": "fixed",
    "top":0,
    "left":0,
    "bottom":0,
    "width": "12rem",
    "padding":"2rem 1rem",
    "background-color": "#F8F9FA",
    "overflow": "scroll",
}
# location of tab information
CONTENT_STYLE = {
    "margin-left": "16rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",}
# Create the Sidebar
sidebar = html.Div(
    [
        html.H2("PfH", className="display-1"),
        html.Hr(),
        html.P ('Home Page'),
        dbc.Nav([
                dbc.NavLink("Background", href="/", active = "exact"),
        ],
            vertical = True,
            pills = True
        ),
        html.Hr(),
        html.P(
            'Partners for Healing', className='lead'
        ),
        dbc.Nav(
            [
                dbc.NavLink("Clinic Usage", href="/clinic_usage", active = "exact"),
                dbc.NavLink("Demographics", href="/demographics", active="exact"),
                dbc.NavLink("Conditions", href="/conditions", active="exact"),
            ],
            vertical = True,
            pills = True,
        ),
        html.Hr(),
        html.P('County', className='lead'),
        dbc.Nav(
            [
                dbc.NavLink("Eligibility", href="/population", active = "exact"),
            ],
            vertical = True,
            pills = True,
        ),
        html.Hr(),
        html.P('Hospital Insights', className='lead'),
        dbc.Nav(
            [
                dbc.NavLink("ER Usage", href="/ER", active = "exact"),
                dbc.NavLink("Rural vs Urban", href="/rural_urban", active="exact"),
                dbc.NavLink("Test 8", href="/test-8", active="exact"),
            ],
            vertical = True,
            pills = True,
        ),
    ],
    style= SIDEBAR_STYLE,
)

# Create the content container
content = html.Div(id="page-content", style=CONTENT_STYLE)

# Define the layout
app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content
])
#---------------------------------------------------------------------------------------------------------------------
# Graphs FOR BACKGROUND
#---------------------------------------------------------------------------------------------------------------------

def county_map():
    #formatting the final shape file
    tn_shape = gpd.read_file('filtered_tn_shapefile.shp')
    tn_shape["geometry"] = (tn_shape.to_crs(tn_shape.estimate_utm_crs()).simplify(100).to_crs(tn_shape.crs))
    tn_shape.set_index('county')
    tn_shape = tn_shape.to_crs(epsg = 4326)
    geojson = tn_shape.__geo_interface__

    #plotting the choropleth
    fig = px.choropleth_mapbox(tn_shape,
                              geojson = tn_shape.geometry,
                              locations = tn_shape.index,
                              color = tn_shape.percent_un,
                               color_discrete_map={'Central West': 'red'},
                              center={"lat": 35.5, "lon": -86},
                               mapbox_style="white-bg", 
                               zoom=5.9,
                              width = 1000,
                              height = 600,
                              custom_data=[tn_shape['county'], tn_shape['population'],tn_shape['percent_un'],tn_shape['uninsured']])

    #editing the hovertemplate
    fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>County Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}%<br>Uninsured Population: %{customdata[3]}')
    #edit the title
    fig.update_layout(title='Percent Uninsured by County in Tennessee')
    #edit the bar label
    fig.update_layout(coloraxis_colorbar_title='Percent Uninsured')

    return fig

#---------------------------------------------------------------------------------------------------------------------
# Graphs FUNCTION FOR CLINIC USAGE
#---------------------------------------------------------------------------------------------------------------------

def monthly_usages(var):
    #Create bar plots for each year
    fig = go.Figure()

    years = monthly_visits['enc year'].unique()
    for year in years:
        df_year = monthly_visits[monthly_visits['enc year'] == var]
        visible = False if year != years[-1] else True
        fig.add_trace(go.Bar(x=df_year['enc month'], y=df_year['patientid'], name=str(year),
                            hovertemplate='Year: %{customdata[0]}<br>' +
                                           'Month: %{x}<br>' +
                                           'Patient Count: %{y}<extra></extra>',
                                            customdata=df_year,
                                            visible=visible))

    #Change layout
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(1, 13)),  # Use a fixed range of values from 1 to 12 for all months
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        ),
        yaxis=dict(
            title='Number of Patients'
        ),
        title='Number of Patients per Month'
    )

    #Plot
    return fig


def patient_count_trends():
    #Create line graph
    fig = go.Figure()

    year_trends['Year'] = year_trends.index.year
    year_trends['Month'] = year_trends.index.strftime('%B')

    fig.add_trace(go.Scatter(
        x=year_trends.index,
        y=year_trends['patientid'],
        mode='lines',
        name='Patient Count',
        hovertemplate='Year: %{customdata[0]}<br>' +
                      'Month: %{customdata[1]}<br>' +
                      'Patient Count: %{y}<extra></extra>',
        customdata=year_trends[['Year', 'Month']]
    ))
    #Update layout
    fig.update_layout(
        xaxis=dict(
            title='Year',
        ),
        yaxis=dict(
            title='Patient Count'
        ),
        title='Patient Count Over Time'
    )
    #Plot
    return fig


def distribution(year):
    #Create histograms
    data = ctrends[ctrends['enc year']==year]
    
    fig = px.histogram(data, x='ptnt  fpl', nbins=10, color='enc year', histnorm='percent')

    #Change layout
    fig.update_layout(
        xaxis=dict(title='Federal Poverty Level'),
        yaxis=dict(title='Percentage'),
        title='Distribution of Federal Poverty Level'
    )

    # Update bar labels
    fig.update_traces(xbins=dict(start=min(bin_ranges), end=250, size=50), overwrite=True)

    # Custom hover template
    fig.update_traces(hovertemplate='Federal Poverty Level: %{x}<br>' +
                                    'Percent: %{y}%<br>',
                      customdata=fig.data[0]['customdata'])

    #Hide legend
    for trace in fig.data:
        trace.update(legendgroup='', showlegend=False)

    #Only show 2023 distribution
    for i in range(0, len(fig.data)-1):
        fig.data[i].visible = False

    #Plot
    return fig


def hear_about():
    #sort the ha ref source counts
    sorted_ha = hear['ref source'].value_counts().sort_values(ascending=False)

    #create the figure
    fig = px.histogram(hear, y='ref source', color='ref source', color_discrete_sequence=px.colors.sequential.Viridis, category_orders={'ref source': sorted_ha.index})

    #edit the hovertemplate
    fig.update_traces(hovertemplate='Reference Source: %{y}<br>Reference Count: %{x}<extra></extra>')


    #edit labels
    fig.update_layout(
        yaxis_title='Reference Source',
        xaxis_title='Reference Count',
        title='How did you hear about Partners for Healing?'
    )

    #plot
    return fig

#---------------------------------------------------------------------------------------------------------------------
# GRAPHS FUNCTIONS FOR DEMOGRAPHICS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# GRAPHS FOR CONDITIONS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# GRAPHS FOR ELIGIBILITY
#---------------------------------------------------------------------------------------------------------------------
def create_line_graph(var):
    filtered_data = gen_census[gen_census['county'].isin(counties_to_show)]
   
    if var == 'uninsured':
        name = 'Uninsured Population by County'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured by County'
        ylabel = "Percent Uninsured"
        
    fig = px.line(
        filtered_data,
        x='year',
        y=var,
        color='county',
        title=name,
        custom_data=['county', 'population', 'percent_uninsured', 'uninsured'],
        color_discrete_sequence=px.colors.qualitative.Plotly
    )

    
    fig.update_layout(
        xaxis_title='Year',  # Set the x-axis title
        yaxis_title=ylabel,  # Set the y-axis title
        title_font=dict(size=20),  # Adjust the title font size
        legend_title='County',  # Set the legend title
        legend_title_font=dict(size=14),  # Adjust the legend title font size
        legend_font=dict(size=12),  # Adjust the legend font size
    )
    
    fig.update_traces(
        mode='lines+markers',
        hovertemplate="<b>%{customdata[0]}</b><br>Year: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Uninsured Population: %{customdata[3]}",
    )
    
    #Set x axis format
    fig.update_layout(xaxis=dict(range=[2018, 2021], tickmode='linear', dtick=1))
    
    return fig



def zip_map(var):   
    #formatting the final shape file
    geojson = zip_shape.__geo_interface__

    #plotting the choropleth
    fig = px.choropleth_mapbox(zip_shape,
                              geojson = zip_shape.geometry,
                              locations = zip_shape.index,
                              color = zip_shape[var],
                               color_discrete_map={'Central West': 'red'},
                              center={"lat": 35.5, "lon": -86.01},
                               mapbox_style="white-bg", 
                               zoom=8,
                              width = 800,
                              height = 650,
                              custom_data=[zip_shape['county_nam'],zip_shape['city'], zip_shape['zip'],zip_shape['population'],zip_shape['percent_un'],zip_shape['uninsured']]
                            )
    if var == 'uninsured':
        name = 'Uninsured Population by Zip Code'
        ylabel = 'Uninsured Population'
        
    else:
        name = 'Percent Uninsured by Zip Code'
        ylabel = "Percent Uninsured"
    
    #editing the hovertemplate
    fig.update_traces(hovertemplate='<b>%{customdata[0]}'+' County'+'</b><br>City: %{customdata[1]}<br>Zip Code: %{customdata[2]}<br>Population: %{customdata[3]}<br>Percent Uninsured: %{customdata[4]}<br>Uninsured Population: %{customdata[5]}')
    #edit the title
    fig.update_layout(title=name)
    #edit the bar label
    fig.update_layout(coloraxis_colorbar_title=ylabel)

    #plot
    return fig

def generate_age_graph(year, var):
    filtered_data = age_census[age_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['age'].isin(['6-18', '19-25', '26-34', '35-44', '45-54', '55-64'])]
    filtered_data = filtered_data[filtered_data['year'] == year]
    years = filtered_data['year'].unique()

    #Create figure
    fig = go.Figure()

    # Create bar plots for each year and county
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True  # Set visibility to False for all bars except the last year
            fig.add_trace(go.Bar(x=county_data['age'],
                                y=county_data[var], name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Age Range: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Population Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))
    if var == 'uninsured':
        name = 'Uninsured Population per County by Age'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Age'
        ylabel = "Percent Uninsured"
    #Update layout with x-axis labels
    fig.update_layout(
       xaxis={'title': 'Age Range'},
        yaxis={'title': ylabel},
        title=name
    )
    return fig

def generate_emp_graph(year,var):
    filtered_data = emp_census[emp_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['employment_status (19-64)'].isin(['employed','unemployed', 'not in labor force'])]
    filtered_data = filtered_data[filtered_data['year']==year]

    years = filtered_data['year'].unique()

    #Create empty figure
    fig = go.Figure()

    #Create bar plots for each year
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True
            fig.add_trace(go.Bar(x=county_data['employment_status (19-64)'], 
                                y=county_data[var],
                                name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Employment Status: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Populaton Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))

    if var == 'uninsured':
        name = 'Uninsured Population per County by Employment Status'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Employent Status'
        ylabel = "Percent Uninsured"
        
    #Update layout with x-axis
    fig.update_layout(
        xaxis={'title': 'Employment Status'},
        yaxis={'title': ylabel},
        title=name
    )
    return fig

def generate_dis_graph(year,var):
    filtered_data = dis_census[dis_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['disability_status'].isin(['disability','no disability'])]
    filtered_data = filtered_data[filtered_data['year']==year]
    years = filtered_data['year'].unique()

    #Create empty figure
    fig = go.Figure()

    #Create bar plots for each year
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True
            fig.add_trace(go.Bar(x=county_data['disability_status'], 
                                y=county_data[var], 
                                name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Disability Status: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Populaton Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))

    if var == 'uninsured':
        name = 'Uninsured Population per County by Disability Status'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Disability Status'
        ylabel = "Percent Uninsured"
    
    #Update layout with x-axis labels
    fig.update_layout(
        xaxis={'title': 'Disability Status'},
        yaxis={'title': ylabel},
        title=name
    )
    return fig

def generate_pov_graph(year,var):
    filtered_data = pov_census[pov_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['poverty_level (%)'].isin(['<138','138-399','>400'])]
    filtered_data = filtered_data[filtered_data['year']==year]

    years = filtered_data['year'].unique()

    #Create empty figure
    fig = go.Figure()

    #Create bar plots for each year
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True
            fig.add_trace(go.Bar(x=county_data['poverty_level (%)'],
                                y=county_data[var],
                                name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Poverty Level: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Populaton Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))
    if var == 'uninsured':
        name = 'Uninsured Population per County by Federal Poverty Level (Percentage)'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Federal Poverty Level (Percentage)'
        ylabel = "Percent Uninsured"
    
    #Update layout with x-axis labels
    fig.update_layout(
        xaxis={'title': 'Poverty Level Percentage'},
        yaxis={'title': ylabel},
        title=name
    )
    return fig






#---------------------------------------------------------------------------------------------------------------------
# BACKGROUND CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# CLINIC USAGE CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------

@app.callback(
    Output("monthly-use", "figure"),
    Input("monthly-use-slider", "value")
)
def update_monthly_usages(var):
    fig = monthly_usages(var)
    return fig

@app.callback(
    Output("poverty-dis", "figure"),
    Input("poverty-dis-slider", "value")
)
def update_distribution(year):
    fig = distribution(year)
    return fig

#---------------------------------------------------------------------------------------------------------------------
# DEMOGRAPHICS CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# CONDITIONS CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------



#---------------------------------------------------------------------------------------------------------------------
# ELIGIBILITY CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------

@app.callback(
    Output("gen-line", "figure"),
    Input("line-dropdown", "value")
)
def update_line_graph(var):
    fig = create_line_graph(var)
    return fig

@app.callback(
    Output("zip-map", "figure"),
    Input("zip-dropdown", "value")
)
def update_zip_map(var):
    fig = zip_map(var)
    return fig

@app.callback(
    Output("age-graph", "figure"),
    [Input("year-slider", "value"),
    Input("age-dropdown", "value")]
)
def update_age_graph(year,var):
    fig = generate_age_graph(year,var)
    return fig

@app.callback(
    Output("emp-graph", "figure"),
    [Input("year-slider-emp", "value"),
    Input("emp-dropdown", "value")]
)
def update_emp_graph(year,var):
    fig = generate_emp_graph(year,var)
    return fig

@app.callback(
    Output("dis-graph", "figure"),
    [Input("year-slider-dis", "value"),
    Input("dis-dropdown", "value")]
)
def update_dis_graph(year,var):
    fig = generate_dis_graph(year,var)
    return fig

@app.callback(
    Output("pov-graph", "figure"),
    [Input("year-slider-pov", "value"),
    Input("pov-dropdown", "value")]
)
def update_pov_graph(year,var):
    fig = generate_pov_graph(year,var)
    return fig


#---------------------------------------------------------------------------------------------------------------------
# ER USE CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# RURAL VS URBAN CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# MAIN CALLBACK AND CALLBACK FUNCTION
#---------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)
def render_page_content(pathname):
    if pathname == "/":
        return [
            html.H1('Bridging The Health Care Coverage Gap', style={'textAlign': 'center'}),
            html.P("Problem Statement"),
            dcc.Graph(id='state-graph',
                      figure=county_map())
        ]
    elif pathname == "/clinic_usage":
        return [
            html.H1("PfH Clinic Usage Throughout Years", style={'textAlign': 'center'}),
            
            dcc.Graph(id='monthly-use'),
            dcc.Slider(
                id="monthly-use-slider",
                min=ctrends['enc year'].min(),
                max=ctrends['enc year'].max(),
                step=1,
                value=ctrends['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            
            html.P('Graph Description^'),
            html.Hr(),
            
            dcc.Graph(id='bargraph',
                       figure=patient_count_trends()),
            html.P('Graph Description^'),
            html.Hr(),
            
            
            dcc.Graph(id='poverty-dis'),
            dcc.Slider(
                id="poverty-dis-slider",
                min=ctrends['enc year'].min(),
                max=ctrends['enc year'].max(),
                step=1,
                value=ctrends['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            
            html.P('Graph Description^'),
            html.Hr(),
            
            dcc.Graph(id='hear-about',
                       figure=hear_about()),
            html.P('Graph Description^'),
            html.Hr(),
        ]
    elif pathname == "/demographics":
        return [
            html.H1("PfH Clinic Demographics", style={'textAlign': 'center'}),
#             dcc.Graph(id='bargraph',
#                       figure=create_line_graph(gen_census, counties_to_show))
        ]
    elif pathname == "/conditions":
        return [
            html.H1("Patient Conditions", style={'textAlign': 'center'}),
#             dcc.Graph(id='bargraph',
#                       figure=create_line_graph(gen_census, counties_to_show))
        ]
    elif pathname == "/population":
        return [
            html.H1("Population of Eligible Patients in Local Counties", style={'textAlign': 'center'}),
            
            html.Hr(),
            html.P('Tab Description'),
            html.Hr(),
            
            dcc.Graph(id='gen-line'),
            dcc.Dropdown(
                id='line-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P('Graph Description^'),
            html.Hr(),
            
            dcc.Graph(id='zip-map'),
            dcc.Dropdown(
                id='zip-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},#must edit values
                    {'label': 'Percent Uninsured', 'value': 'percent_un'},
                ],
                value='percent_un'
            ),
            html.P(''),
            html.P('Graph Description^'),
            html.Hr(),
            
            
            #Age
            dcc.Graph(id='age-graph'),
            dcc.Slider(
                id="year-slider",
                min=emp_census['year'].min(),
                max=emp_census['year'].max(),
                step=1,
                value=emp_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2023)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='age-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P('Graph Description^'),
            html.Hr(),
            

            #Employment
            dcc.Graph(id='emp-graph'),
            dcc.Slider(
                id="year-slider-emp",
                min=emp_census['year'].min(),
                max=emp_census['year'].max(),
                step=1,
                value=emp_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2023)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='emp-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P('Graph Description^'),
            html.Hr(),
            
            #Disability
            dcc.Graph(id='dis-graph'),
            dcc.Slider(
                id="year-slider-dis",
                min=dis_census['year'].min(),
                max=dis_census['year'].max(),
                step=1,
                value=dis_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2022)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='dis-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P('Graph Description^'),
            html.Hr(),
            
            #Poverty Section
            dcc.Graph(id='pov-graph'),
            dcc.Slider(
                id="year-slider-pov",
                min=pov_census['year'].min(),
                max=pov_census['year'].max(),
                step=1,
                value=pov_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2022)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='pov-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P('Graph Description^'),
            html.Hr(),
            ]

    
    elif pathname == "/ER":
        return [
            html.H1("Preventable ER Visits", style={'textAlign': 'center'}),
#             dcc.Graph(id='bargraph',
#                       figure=create_line_graph(gen_census, counties_to_show))
        ]
    elif pathname == "/rural_urban":
        return [
            html.H1("Rural vs Urban Healthcare", style={'textAlign': 'center'}),
#             dcc.Graph(id='bargraph',
#                       figure = create_line_graph(gen_census, counties_to_show) #added yearly uninsured graph)
                        ]
# Define callbacks to update the content based on the selected sub-tabs
# Run the application
if __name__=='__main__':
    app.run_server(debug=True, port=8010)
#http://127.0.0.1:8002/county